# Create and Advanced Custom Player

All the imports are centralized in the cell below.

In [1]:
from maverick import (
    Game,
    Player,
    ActionType,
    GameState,
    PlayerLike,
    Street,
    HandType,
    PlayerAction,
)
from maverick.players import FoldBot
from maverick.utils import estimate_strongest_hand, score_hand
import logging
import sys

## Custom Players

In [2]:
class AggressiveBot(Player):
    """An aggressive bot that frequently bets and raises."""

    def decide_action(
        self,
        game_state: GameState,
        valid_actions: list[ActionType],
        min_raise: int
    ) -> PlayerAction:
        """Bet or raise aggressively."""
        # Try to raise if possible
        if ActionType.RAISE in valid_actions:
            raise_amount = min_raise
            if raise_amount <= self.stack:
                return PlayerAction(player_id=self.id, action_type=ActionType.RAISE, raise_amount=raise_amount)

        # Otherwise bet if possible
        if ActionType.BET in valid_actions:
            bet_amount = game_state.big_blind * 2
            if bet_amount <= self.stack:
                return PlayerAction(player_id=self.id, action_type=ActionType.BET, bet_amount=bet_amount)

        # Call if we can't bet/raise
        if ActionType.CALL in valid_actions:
            call_amount = game_state.current_bet - self.current_bet
            if call_amount <= self.stack:
                return PlayerAction(player_id=self.id, action_type=ActionType.CALL, call_amount=call_amount)

        # Check if possible
        if ActionType.CHECK in valid_actions:
            return PlayerAction(player_id=self.id, action_type=ActionType.CHECK, check_amount=0)
        
        # Otherwise fold
        return PlayerAction(player_id=self.id, action_type=ActionType.FOLD, fold_amount=0)

In [3]:
class ComplexPlayer(Player):
    """A custom player with a complex strategy."""

    def decide_action(
        self,
        game_state: GameState,
        valid_actions: list[ActionType],
        min_raise: int
    ) -> PlayerAction:
        print(f"\n{self.name} is requested to take action.")
        
        # Information related to the game state
        print("     Current street:", game_state.street)
        print("     Current pot:", game_state.pot)
        print("     Small blind:", game_state.small_blind)
        print("     Big blind:", game_state.big_blind)
        print("     Community cards:", game_state.community_cards)
        
        # Information related to the player
        print("     Player stack:", self.stack)
        print("     Current player bet:", self.current_bet)
        print("     Private cards:", self.holding.cards)
        
        # Information related to the available actions
        print("     Valid actions:", valid_actions)
        print("     Minimum raise:", min_raise)
        
        # Private and community cards
        private_cards = self.holding.cards
        community_cards = game_state.community_cards
        print("     Private cards:", private_cards)
        print("     Community cards:", community_cards)
        
        # Estimate the strongest possible hand of the player, considering
        # all community cards.
        strongest_hand, strongest_hand_prob = estimate_strongest_hand(
            private_cards,
            community_cards,
            n_min_private=0,
            n_simulations=1000,
            n_players=len(game_state.get_players_in_hand())
        )
        strongest_hand_type, strongest_hand_score = score_hand(strongest_hand)
        print("     Strongest hand:", strongest_hand)
        print("     Strongest hand type:", strongest_hand_type.name)
        print("     Strongest hand score:", strongest_hand_score)
        print("     Estimated strength:", strongest_hand_prob)
        
        # This is the players strategy
        match game_state.street:
            case Street.PRE_FLOP:
                if strongest_hand_prob < 0.2:
                    # This is the time to bluff if you want to
                    print("     Player decides to FOLD.\n")
                    return PlayerAction(player_id=self.id, action_type=ActionType.FOLD)
                elif strongest_hand_prob > 0.4 and strongest_hand_type >= HandType.PAIR:
                    if ActionType.BET in valid_actions:
                        bet_amount = max(game_state.big_blind * 4, min_raise)
                        print(f"     Player decides to BET {bet_amount}.\n")
                        return PlayerAction(player_id=self.id, action_type=ActionType.BET, amount=bet_amount)
                    elif ActionType.RAISE in valid_actions:
                        raise_amount = max(game_state.big_blind * 4, min_raise)
                        print(f"     Player decides to RAISE {raise_amount}.\n")
                        return PlayerAction(player_id=self.id, action_type=ActionType.RAISE, amount=raise_amount)
                    elif ActionType.CALL in valid_actions:
                        call_amount = max(game_state.current_bet - self.current_bet, self.stack)
                        print(f"     Player decides to CALL {call_amount}.\n")
                        return PlayerAction(player_id=self.id, action_type=ActionType.CALL, amount=call_amount)
                    elif ActionType.CHECK in valid_actions:
                        print("     Player decides to CHECK.\n")
                        return PlayerAction(player_id=self.id, action_type=ActionType.CHECK)
                    else:
                        print("     Player decides to ALL IN.\n")
                        return PlayerAction(player_id=self.id, action_type=ActionType.ALL_IN, amount=self.stack)           
                else:
                    if ActionType.CHECK in valid_actions:
                        print("     Player decides to CHECK.\n")
                        return PlayerAction(player_id=self.id, action_type=ActionType.CHECK)
                    else:
                        print("     Player decides to FOLD.\n")
                        return PlayerAction(player_id=self.id, action_type=ActionType.FOLD)
            
            case Street.FLOP:
                if strongest_hand_type in [
                    HandType.ROYAL_FLUSH, 
                    HandType.STRAIGHT_FLUSH, 
                    HandType.FOUR_OF_A_KIND, 
                    HandType.FULL_HOUSE
                ]:
                    if ActionType.CHECK in valid_actions:
                        print("     Player decides to CHECK.\n")
                        return PlayerAction(player_id=self.id, action_type=ActionType.CHECK)
                    assert ActionType.CALL in valid_actions
                    call_amount = max(game_state.current_bet - self.current_bet, self.stack)
                    print(f"     Player decides to CALL {call_amount}.\n")
                    return PlayerAction(player_id=self.id, action_type=ActionType.CALL, amount=call_amount)
                else:
                    if strongest_hand_prob < 0.3:
                        print("     Player decides to FOLD.\n")
                        return PlayerAction(player_id=self.id, action_type=ActionType.FOLD)
                    elif strongest_hand_prob > 0.6:
                        if ActionType.RAISE in valid_actions:
                            raise_amount = max(game_state.big_blind * 3, min_raise)
                            print(f"     Player decides to RAISE {raise_amount}.\n")
                            return PlayerAction(player_id=self.id, action_type=ActionType.RAISE, amount=raise_amount)
                        elif ActionType.CALL in valid_actions:
                            call_amount = max(game_state.current_bet - self.current_bet, self.stack)
                            print(f"     Player decides to CALL {call_amount}.\n")
                            return PlayerAction(player_id=self.id, action_type=ActionType.CALL, amount=call_amount)
                    else:
                        if ActionType.CHECK in valid_actions:
                            print("     Player decides to CHECK.\n")
                            return PlayerAction(player_id=self.id, action_type=ActionType.CHECK)
                        else:
                            print("     Player decides to FOLD.\n")
                            return PlayerAction(player_id=self.id, action_type=ActionType.FOLD)
                        
            case Street.TURN:
                to_call = max(0, game_state.current_bet - self.current_bet)

                # Very strong made hands: play safely (slowplay), but don't fold.
                if strongest_hand_type in [
                    HandType.ROYAL_FLUSH,
                    HandType.STRAIGHT_FLUSH,
                    HandType.FOUR_OF_A_KIND,
                    HandType.FULL_HOUSE,
                ]:
                    if ActionType.CHECK in valid_actions:
                        print("     Player decides to CHECK.\n")
                        return PlayerAction(player_id=self.id, action_type=ActionType.CHECK)
                    if ActionType.CALL in valid_actions:
                        call_amount = min(to_call, self.stack)
                        print(f"     Player decides to CALL {call_amount}.\n")
                        return PlayerAction(player_id=self.id, action_type=ActionType.CALL, amount=call_amount)
                # Weak: avoid putting more money in.
                if strongest_hand_prob < 0.25:
                    if ActionType.CHECK in valid_actions:
                        print("     Player decides to CHECK.\n")
                        return PlayerAction(player_id=self.id, action_type=ActionType.CHECK)
                    print("     Player decides to FOLD.\n")
                    return PlayerAction(player_id=self.id, action_type=ActionType.FOLD)

                # Strong: value bet / raise.
                if strongest_hand_prob > 0.65:
                    if ActionType.BET in valid_actions:
                        bet_amount = min(self.stack, max(game_state.big_blind * 3, min_raise))
                        print(f"     Player decides to BET {bet_amount}.\n")
                        return PlayerAction(player_id=self.id, action_type=ActionType.BET, amount=bet_amount)
                    if ActionType.RAISE in valid_actions:
                        raise_amount = min(self.stack, max(game_state.big_blind * 3, min_raise))
                        print(f"     Player decides to RAISE {raise_amount}.\n")
                        return PlayerAction(player_id=self.id, action_type=ActionType.RAISE, amount=raise_amount)
                    if ActionType.CALL in valid_actions:
                        call_amount = min(to_call, self.stack)
                        print(f"     Player decides to CALL {call_amount}.\n")
                        return PlayerAction(player_id=self.id, action_type=ActionType.CALL, amount=call_amount)
                # Medium: prefer checking, otherwise call small commitments.
                if ActionType.CHECK in valid_actions:
                    print("     Player decides to CHECK.\n")
                    return PlayerAction(player_id=self.id, action_type=ActionType.CHECK)
                if ActionType.CALL in valid_actions:
                    call_amount = min(to_call, self.stack)
                    print(f"     Player decides to CALL {call_amount}.\n")
                    return PlayerAction(player_id=self.id, action_type=ActionType.CALL, amount=call_amount)
                print("     Player decides to FOLD.\n")
                return PlayerAction(player_id=self.id, action_type=ActionType.FOLD)

            case Street.RIVER:
                to_call = max(0, game_state.current_bet - self.current_bet)

                # On the river, be even more conservative with weak hands.
                if strongest_hand_prob < 0.35:
                    if ActionType.CHECK in valid_actions:
                        print("     Player decides to CHECK.\n")
                        return PlayerAction(player_id=self.id, action_type=ActionType.CHECK)
                    print("     Player decides to FOLD.\n")
                    return PlayerAction(player_id=self.id, action_type=ActionType.FOLD)

                # Value bet thin only when strong.
                if strongest_hand_prob > 0.75:
                    if ActionType.BET in valid_actions:
                        bet_amount = min(self.stack, max(game_state.big_blind * 3, min_raise))
                        print(f"     Player decides to BET {bet_amount}.\n")
                        return PlayerAction(player_id=self.id, action_type=ActionType.BET, amount=bet_amount)
                    if ActionType.RAISE in valid_actions:
                        raise_amount = min(self.stack, max(game_state.big_blind * 3, min_raise))
                        print(f"     Player decides to RAISE {raise_amount}.\n")
                        return PlayerAction(player_id=self.id, action_type=ActionType.RAISE, amount=raise_amount)
                    if ActionType.CALL in valid_actions:
                        call_amount = min(to_call, self.stack)
                        print(f"     Player decides to CALL {call_amount}.\n")
                        return PlayerAction(player_id=self.id, action_type=ActionType.CALL, amount=call_amount)
                # Medium strength: check if possible, otherwise call.
                if ActionType.CHECK in valid_actions:
                    print("     Player decides to CHECK.\n")
                    return PlayerAction(player_id=self.id, action_type=ActionType.CHECK)
                if ActionType.CALL in valid_actions:
                    call_amount = min(to_call, self.stack)
                    print(f"     Player decides to CALL {call_amount}.\n")
                    return PlayerAction(player_id=self.id, action_type=ActionType.CALL, amount=call_amount)
                print("     Player decides to FOLD.\n")
                return PlayerAction(player_id=self.id, action_type=ActionType.FOLD)

        # Make sure to always return a valid action. Folding is always available.
        print("     Player decides to FOLD.\n")
        sys.stdout.flush()  # Ensure all prints are output before returning
        return PlayerAction(player_id=self.id, action_type=ActionType.FOLD)

In [4]:
# Configure logging such that we only get the log messages of the game
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
)
logging.getLogger().setLevel(logging.WARNING)  # Set root logger to WARNING
logging.getLogger("maverick").setLevel(logging.DEBUG)  # Only maverick logs at DEBUG

# Redirect maverick logs to stdout, so they appear in the correct event order
handler = logging.StreamHandler(sys.stdout)
logging.getLogger("maverick").handlers = [handler]
logging.getLogger("maverick").propagate = False

# Create game with blinds
game = Game(small_blind=10, big_blind=20, max_hands=40)

# Create and add players with different strategies
players: list[PlayerLike] = [
    ComplexPlayer(id="p1", name="ComplexPlayer", stack=1000, seat=0),
    FoldBot(id="p2", name="FoldBot", stack=1000, seat=1),
    AggressiveBot(id="p3", name="AggressiveBot", stack=1000, seat=2),
]

for player in players:
    game.add_player(player)
    
game.start()

Player ComplexPlayer joined the game.
Player FoldBot joined the game.
Player AggressiveBot joined the game.
Game started.

============================== Hand 1 ==============================

PRE_FLOP | Dealing hole cards. Button: ComplexPlayer
PRE_FLOP | Posting small blind of 10 by player FoldBot. Remaining stack: 990
PRE_FLOP | Posting big blind of 20 by player AggressiveBot. Remaining stack: 980

ComplexPlayer is requested to take action.
     Current street: Street.PRE_FLOP
     Current pot: 30
     Small blind: 10
     Big blind: 20
     Community cards: []
     Player stack: 1000
     Current player bet: 0
     Private cards: [9♦, 5♥]
     Valid actions: [<ActionType.FOLD: 1>, <ActionType.CALL: 3>, <ActionType.RAISE: 5>, <ActionType.ALL_IN: 6>]
     Minimum raise: 40
     Private cards: [9♦, 5♥]
     Community cards: []
     Strongest hand: [9♦, 5♥]
     Strongest hand type: HIGH_CARD
     Strongest hand score: 100.0905
     Estimated strength: 0.302
     Player decides to FOLD

/Users/baloghbence/Documents/Projects/maverick/src/maverick/game.py:391: UserWarning: Player AggressiveBot action invalid, folding.
  warn(f"Player {current_player.name} action invalid, folding.")


     Strongest hand: [6♣, 7♥]
     Strongest hand type: HIGH_CARD
     Strongest hand score: 100.0706
     Estimated strength: 0.262
     Player decides to FOLD.

PRE_FLOP | Player ComplexPlayer folds.
PRE_FLOP | Current pot: 30
PRE_FLOP | Player FoldBot folds.
PRE_FLOP | Current pot: 30
PRE_FLOP | Betting round complete

SHOWDOWN | Player AggressiveBot wins 30 from the pot.
SHOWDOWN | Showdown complete

Hand ended

============================== Hand 5 ==============================

PRE_FLOP | Dealing hole cards. Button: FoldBot
PRE_FLOP | Posting small blind of 10 by player AggressiveBot. Remaining stack: 1000
PRE_FLOP | Posting big blind of 20 by player ComplexPlayer. Remaining stack: 980
PRE_FLOP | Player FoldBot folds.
PRE_FLOP | Current pot: 30
PRE_FLOP | Player AggressiveBot intended to take action: player_id='p3' action_type=<ActionType.RAISE: 5> amount=None.
PRE_FLOP | Player AggressiveBot action invalid, folding.
Traceback (most recent call last):
  File "/Users/baloghbence/

In [5]:
for player in players:
    print(f"{player.name} - Stack: {player.stack}")

ComplexPlayer - Stack: 1060
FoldBot - Stack: 990
AggressiveBot - Stack: 950


## Best Practices

- Always end `decide_action` with a FOLD action. Folding is always available as an action and it ensures a feasible decision among all circumstances.
- Use as much information as possible.